In [35]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [36]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [37]:
df = pd.read_csv(f'artifacts/raw_data/customer_churn_removed_col.csv')

In [38]:
df.isnull().sum()

Age                           0
Gender                        0
Location                      0
Subscription_Length_Months    0
Monthly_Bill                  0
Total_Usage_GB                0
Churn                         0
dtype: int64

## Transformation

In [39]:
df['Gender']=df['Gender'].replace({'Male':0,'Female':1})

In [40]:
X = df.drop(columns='Churn')
y = df['Churn']

In [41]:
cat_cols =[feature for feature in df.columns if df[feature].dtype == 'O']
num_cols =[feature for feature in df.columns if df[feature].dtype != 'O']
print(cat_cols)
print(num_cols)

['Location']
['Age', 'Gender', 'Subscription_Length_Months', 'Monthly_Bill', 'Total_Usage_GB', 'Churn']


In [42]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (OneHotEncoder,
                                   MinMaxScaler)
import numpy as np
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.30, random_state=42)

In [44]:
df['Gender']=df['Gender'].replace({'Male':0,'Female':1})

In [45]:
X_train

,Age,Gender,Location,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB
76513,25,0,Los Angeles,13,98.13,468
60406,54,1,Houston,2,71.90,383
27322,32,0,Chicago,20,65.65,427
53699,61,1,Houston,12,83.20,64
65412,70,1,New York,5,41.93,99
...,...,...,...,...,...,...
6265,35,0,Miami,21,67.33,235
54886,56,0,Chicago,13,85.40,347
76820,69,0,Houston,2,76.24,321
860,55,0,Chicago,12,89.19,315


In [46]:
preprocessing = ColumnTransformer(transformers=[
    ('OHE',OneHotEncoder(drop='first',sparse=False,dtype=np.int64),['Location']),
    # ('scaling',MinMaxScaler(),['Age', 'Subscription_Length_Months', 'Monthly_Bill', 'Total_Usage_GB'])
],remainder='passthrough')

In [47]:
X_train= preprocessing.fit_transform(X_train)
X_test = preprocessing.transform(X_test)

In [48]:
from sklearn.linear_model import LogisticRegression


logst = LogisticRegression()
logst.fit(X_train,y_train)

LogisticRegression()

In [49]:
def evaluate(true,pred):
    
    cm = confusion_matrix(true, pred)
    accuracy = accuracy_score(true, pred)
    recall = recall_score(true, pred)
    precision = precision_score(true, pred)
    
    report = classification_report(true, pred)

    evaluation_report = {
        'confusion_matrix': cm,
        'accuracy': accuracy,
        'recall': recall,
        'precision': precision,
        'classification_report': report
    }
    print('confusion metrics: \n',cm)
    print('accuracy: \n',accuracy)
    print('precision: \n',precision)
    print('recall: \n',recall)
    print('classification_report: \n',report)

## model

In [50]:
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, classification_report

In [51]:
def evaluate(true,pred):
    
    cm = confusion_matrix(true, pred)
    accuracy = accuracy_score(true, pred)
    recall = recall_score(true, pred)
    precision = precision_score(true, pred)
    
    report = classification_report(true, pred)

    evaluation_report = {
        'confusion_matrix': cm,
        'accuracy': accuracy,
        'recall': recall,
        'precision': precision,
        'classification_report': report
    }
    print('confusion metrics: \n',cm)
    print('accuracy: \n',accuracy)
    print('precision: \n',precision)
    print('recall: \n',recall)
    print('classification_report: \n',report)
    
    # return evaluation_report


    

## training all aglorithms

In [52]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

In [53]:
def evaluate(true,pred):
    accuracy = accuracy_score(true, pred)
    recall = recall_score(true, pred)
    precision = precision_score(true, pred)
    return accuracy , recall , precision


In [54]:
params = {
    "Gradient Boosting Classifier": {
        'loss': ['deviance', 'exponential'],  # Classification loss functions
        'learning_rate': [0.1, 0.01, 0.05, 0.001],
        'criterion': ['friedman_mse'],
        'n_estimators': [8, 16, 32, 64, 128, 256],
        'subsample': [0.6, 0.7, 0.75, 0.8, 0.85, 0.9],
        'max_features': ['auto', 'sqrt', 'log2']
    },
    "XGBoost Classifier": {
        'learning_rate': [0.1, 0.01, 0.05, 0.001],
        'n_estimators': [8, 16, 32, 64, 128, 256],
        'subsample': [0.6, 0.7, 0.75, 0.8, 0.85, 0.9],
        'max_depth': [3, 4, 5, 6, 7, 8],
        'min_child_weight': [1, 2, 3, 4]
    },
    "CatBoost Classifier": {
        'depth': [6, 8, 10],
        'learning_rate': [0.01, 0.05, 0.1],
        'iterations': [30, 50, 100],
        'loss_function': ['Logloss', 'CrossEntropy'],
        'eval_metric': ['Logloss', 'AUC'],
    },
    "AdaBoost Classifier": {
        'learning_rate': [0.1, 0.01, 0.5, 0.001],
        'n_estimators': [8, 16, 32, 64, 128, 256],
        'algorithm': ['SAMME', 'SAMME.R'],
    },
    "Random Forest Classifier": {
        'n_estimators': [8, 16, 32, 64, 128, 256],
        'criterion': ['gini', 'entropy'],
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt', 'log2']
    }
}


In [55]:
models = {
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "XGBoost Classifier": XGBClassifier(),
    "CatBoost Classifier": CatBoostClassifier(),
    "AdaBoost Classifier": AdaBoostClassifier(),
    "Random Forest Classifier": RandomForestClassifier()
}


In [56]:

class ModelTraining:
    def __init__(self):
        self.model_list = []
        self.accuracy_list = []
        self.best_params_dict = {}

    def train_model(self, models, params, X_train, y_train, X_test, y_test):
        for i, model_name in enumerate(models.keys()):
            model = models[model_name]
            param = params[model_name]

            grid_search_cv = RandomizedSearchCV(estimator=model, param_distributions=param, cv=5)
            grid_search_cv.fit(X_train, y_train)

            best_params = grid_search_cv.best_params_
            model.set_params(**best_params)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            accuracy, recall, precision = evaluate(true=y_test, pred=y_pred)

            print(model_name)
            self.model_list.append(model_name)

            print("- precision : {:.4f}".format(precision))
            print("- recall : {:.4f}".format(recall))
            print("- accuracy : {:.4f}".format(accuracy))
            print('\n')

            self.best_params_dict[model_name] = best_params

            self.accuracy_list.append(accuracy)

    def return_accuracy_list(self):
        return self.accuracy_list
    
    def best_params(self):
        return self.best_params_dict


In [57]:
train_model = ModelTraining()

In [58]:
train_model.train_model(models=models,params=params,X_train=X_train,y_train=y_train,X_test=X_test,y_test=y_test)

Gradient Boosting Classifier
- precision : 0.4965
- recall : 0.4600
- accuracy : 0.5019


XGBoost Classifier
- precision : 0.4929
- recall : 0.4745
- accuracy : 0.4983


0:	learn: 0.6928007	total: 172ms	remaining: 8.42s
1:	learn: 0.6924396	total: 192ms	remaining: 4.61s
2:	learn: 0.6921935	total: 215ms	remaining: 3.37s
3:	learn: 0.6918131	total: 236ms	remaining: 2.71s
4:	learn: 0.6915185	total: 257ms	remaining: 2.31s
5:	learn: 0.6911303	total: 281ms	remaining: 2.06s
6:	learn: 0.6907186	total: 302ms	remaining: 1.85s
7:	learn: 0.6903528	total: 322ms	remaining: 1.69s
8:	learn: 0.6899968	total: 344ms	remaining: 1.56s
9:	learn: 0.6895556	total: 364ms	remaining: 1.46s
10:	learn: 0.6892014	total: 385ms	remaining: 1.37s
11:	learn: 0.6889222	total: 411ms	remaining: 1.3s
12:	learn: 0.6886489	total: 432ms	remaining: 1.23s
13:	learn: 0.6881600	total: 453ms	remaining: 1.16s
14:	learn: 0.6878195	total: 474ms	remaining: 1.11s
15:	learn: 0.6874908	total: 495ms	remaining: 1.05s
16:	learn: 0.6871495	tota

In [59]:
train_model.best_params_dict

{'Gradient Boosting Classifier': {'subsample': 0.7,
  'n_estimators': 64,
  'max_features': 'log2',
  'loss': 'exponential',
  'learning_rate': 0.1,
  'criterion': 'friedman_mse'},
 'XGBoost Classifier': {'subsample': 0.6,
  'n_estimators': 64,
  'min_child_weight': 1,
  'max_depth': 7,
  'learning_rate': 0.1},
 'CatBoost Classifier': {'loss_function': 'CrossEntropy',
  'learning_rate': 0.1,
  'iterations': 100,
  'eval_metric': 'Logloss',
  'depth': 8},
 'AdaBoost Classifier': {'n_estimators': 16,
  'learning_rate': 0.01,
  'algorithm': 'SAMME.R'},
 'Random Forest Classifier': {'n_estimators': 256,
  'min_samples_split': 10,
  'min_samples_leaf': 2,
  'max_features': 'sqrt',
  'max_depth': 40,
  'criterion': 'entropy'}}

In [60]:
model_accuracy=pd.DataFrame(list(zip(list(models.keys()),train_model.accuracy_list)),columns=['model','accuracy'])
model_accuracy

,model,accuracy
0,Gradient Boosting Classifier,0.501867
1,XGBoost Classifier,0.498333
2,CatBoost Classifier,0.499667
3,AdaBoost Classifier,0.503067
4,Random Forest Classifier,0.498000
